In [86]:
from typing import Tuple, List
import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam import CoGroupByKey, PCollection
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [87]:
p = beam.Pipeline(InteractiveRunner())

In [88]:
m: PCollection[Tuple[str, int]] = p | "M" >> beam.Create([('a', 1), ('b', 2), ('b', 3)])
ib.collect(m)

,0,1
0,a,1
1,b,2
2,b,3


In [89]:
s = p | "S" >> beam.Create([('b', 10), ('c', 20)])
ib.collect(s)

,0,1
0,b,10
1,c,20


In [93]:
cogbk: PCollection[Tuple[str, Tuple[List[int], List[int]]]] = (m, s) | CoGroupByKey()
ib.collect(cogbk)

,0,1
0,b,"([2, 3], [10])"
1,c,"([], [20])"
2,a,"([1], [])"


In [59]:
class MySum(beam.DoFn):
    def process(self, element: Tuple[str, Tuple[List[int], List[int]]], *args, **kwargs):
        key: str = element[0]
        _sum: int = 0
        val: Tuple[List[int], List[int]] = element[1]
        for _list in val:
            _l: List[int] = _list
            _sum+=sum(_l)
        yield key, _sum

In [96]:
join_and_sum: PCollection[Tuple[str, int]] = cogbk | beam.ParDo(MySum())
ib.collect(join_and_sum)

,0,1
0,b,15
1,c,20
2,a,1
